Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru. Для парсинга использовать XPath. Структура данных должна содержать:
название источника;
наименование новости;
ссылку на новость;
дата публикации.
Сложить собранные новости в БД

In [92]:
pip install lxml

In [12]:
from lxml import html

In [94]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [13]:
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError

In [14]:
import requests
from pprint import pprint
import pandas as pd 
from lxml import html

In [36]:
def add_to_db(doc, only_new):
    
    exists = news.find_one({'link':doc['link']})
    if only_new == True:
        if not exists:
            try:
                news.insert_one(doc)
            except DuplicateKeyError:
                pass
    else:
        try:
            news.insert_one(doc)
        except DuplicateKeyError:
            pass

In [16]:
client = MongoClient('localhost', 27017)

In [17]:
db = client['lentanews']

In [18]:
news= db.news # item collection

In [ ]:
#Получаем данные со страницы

In [19]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}

In [20]:
url = 'https://lenta.ru'

In [21]:
session = requests.Session()

In [22]:
response = session.get(url,headers = headers)

In [23]:
response

<Response [200]>

In [24]:
dom = html.fromstring(response.text)

В данном случае забирала главные новости

In [27]:
items = dom.xpath("//div[contains(@class,'last24')]//a[contains(@class,'card-mini')]")

In [29]:
r = input('Введите Y, если в базу нужно добавлять только новые новости, любое другое значение если добавлять все: ')
r=r.strip()
r=r.lower()
if r =='y'or'Y':
    only_new=True
else:
    only_new=False

Введите Y, если в базу нужно добавлять только новые новости, любое другое значение если добавлять все: y


In [30]:
only_new

True

In [43]:
for item in items:
    itnews={}
    name = item.xpath(".//h3[contains(@class,'card-mini__title')]/text()")
    link =item.xpath("./@href")
    dat = str(link)
    dat = dat[8:18:1]
 
    
    link=url+''.join(link)
    itnews['resource']=url #источник
    itnews['name']=name
    itnews['link']=link
    itnews['date']=dat
    
    add_to_db(itnews, only_new) #вызов функции для добавления значений в базу

In [42]:
for item in news.find({'resource':url}):
    pprint(item)

{'_id': ObjectId('64c987613e9ac2aa71c61708'),
 'date': '2023/08/01',
 'link': 'https://lenta.ru/news/2023/08/01/fraza/',
 'name': ['На Западе заявили об указавшем Зеленскому его место одной фразой '
          'Байдене'],
 'resource': 'https://lenta.ru'}
{'_id': ObjectId('64c987623e9ac2aa71c61709'),
 'date': '2023/08/01',
 'link': 'https://lenta.ru/news/2023/08/01/niger_uranium/',
 'name': ['Захватившая власть в Нигере хунта запретила поставки урана во '
          'Францию. Как это отразится на ЕС и санкциях против России?'],
 'resource': 'https://lenta.ru'}
{'_id': ObjectId('64c987623e9ac2aa71c6170a'),
 'date': '2023/08/01',
 'link': 'https://lenta.ru/news/2023/08/01/control/',
 'name': ['В ДНР сообщили о контроле российских сил над Клещеевкой'],
 'resource': 'https://lenta.ru'}
{'_id': ObjectId('64c987623e9ac2aa71c6170b'),
 'date': '2023/08/01',
 'link': 'https://lenta.ru/news/2023/08/01/bespilot/',
 'name': ['Украинские беспилотники вновь попытались атаковать Москву. Что '
          